# Quantitative Value Strategy
    The Strategy was discovered and pioneered by Tobias Carlisle and Wes Grey's Book, "Quantitative Value: A Practiotioner's Guide to Automating Intelligent Investment and Eliminating Behaviorial Errors". It involves picking stocks that appear to be trading less than their intrinsic value. The strategy earns very high returns with relatively less volatility. It computes the Valuation and Quality of Stocks and ranks/sorts them in order to select a stock that will yield the best returns. This is one type of strategy out of the many Value Investing Strategies.
    
    This project will focus on building an investment strategy that selects the 50 stocks with the best value metrics and will then calculate recommended trades for an Equal Weight portfolio of the 50 stocks.

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing the List of Stocks and API Token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making the First API Call
I will be making the first API Call using Apple's Stock `AAPL` and this will generate a JSON file of Apple's stock information (P/E Ratio, Latest Price, marketCap, etc.)

In [97]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 77939924, 'calculationPrice': 'iexlasttrade', 'change': -0.05, 'changePercent': -0.00035, 'close': 0, 'closeSource': 'foiailcf', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 151.66, 'iexCloseTime': 1673693213143, 'iexLastUpdated': 1698446792223, 'iexMarketPercent': 0.01913818304962516, 'iexOpen': 154.71, 'iexOpenTime': 1700731829569, 'iexRealtimePrice': 153.41, 'iexRealtimeSize': 1, 'iexVolume': 974998, 'lastTradeTime': 1655383549441, 'latestPrice': 152.46, 'latestSource': 'IEX Last Trade', 'latestTime': 'October 25, 2021', 'latestUpdate': 1693526384666, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2564527958243, 'oddLotDelayedPrice

## Parsing the API Call
I want to identify the specific metric we need - price-to-earnings ratio.
* Here is how the data is parsed to retrive Apple's Latest Price & P/E Ratio:

In [98]:
price = data['latestPrice']
peRatio = data['peRatio']
print("Latest Price of AAPL: ", price)
print("Price-to-Earning Ratio of AAPL: ", peRatio)

Latest Price of AAPL:  152.46
Price-to-Earning Ratio of AAPL:  30.57


## Executing a Batch API Call & Building Our DataFrame
By executing Batch API Calls we will be able to build our Dataframe that will consist of the Stock's Symbol, Price, P/E Ratio, and the Number of Shares to Buy


In [6]:
# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# Must split up the 500 stock symbols into subgroups for better API Performance
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
# Subgroups created here
symbol_groups = list(chunks(stocks['Symbol'], 100))

#Subgroups are then combined into a list of 5 stock groups
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns = ['Symbol', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']


Here we have created an empty DataFrame that has headers, `Symbol`, `Price`, `Price-to-Earnings Ratio`, and `Number of Shares to Buy`.

In [99]:
QV_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        QV_dataframe = QV_dataframe.append(
        pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
QV_dataframe

,Symbol,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,165.75,51.36,N/A
1,AAL,19.62,-3.95,N/A
2,AAP,231.32,25.08,N/A
3,AAPL,154.08,29.46,N/A
4,ABBV,110.80,30.06,N/A
...,...,...,...,...
500,YUM,127.78,29.42,N/A
501,ZBH,151.14,34.5,N/A
502,ZBRA,534.58,38.22,N/A
503,ZION,67.86,6.76,N/A


## Removing Glamour Stocks
“A popular stock characterized by high earnings growth rate and a price that rise is faster than the market average in a bull market.” (https://marketbusinessnews.com/financial-glossary/glamour-stocks/)

Glamour stocks do not add any significance to our strategy. They are the opposite of "value" stocks. The next step is to remove the glamour stocks from the dataframe. Sort the dataframe by P/E Ratio, and keep the top 50.

In [11]:
#Sorted the stocks such that the stocks with the lowest PE Ratio are at the top
#inplace = true was used b/c the original data frame was modified and it didn't just return a modified data frame
QV_dataframe.sort_values('Price-to-Earnings Ratio',inplace = True)

# All rows that had a PE Ratio below 0 were dropped from data frame
QV_dataframe = QV_dataframe[QV_dataframe['Price-to-Earnings Ratio'] > 0]

#Only include the stocks that are the 50 lowest PE ratios
QV_dataframe = QV_dataframe[:50]

#Reset the Index
QV_dataframe.reset_index(inplace = True)

#Stopped the new index column that was created from resetting the Index
QV_dataframe.drop(['index'], axis = 1, inplace = True)


In [12]:
QV_dataframe

,level_0,Symbol,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,0,EBAY,84.57,4.37,N/A
1,1,OGN,36.43,4.4,N/A
2,2,BIO,786.88,5.71,N/A
3,3,MPC,67.52,5.77,N/A
4,4,PRU,116.80,6.31,N/A
5,5,GM,59.02,6.69,N/A
6,6,ZION,69.58,6.78,N/A
7,7,COF,170.07,6.8,N/A
8,8,AFL,58.47,7,N/A
9,9,CINF,128.06,7.05,N/A


## Calculating the Number of Shares to Buy

Calculating the number of shares to buy is found by finding the amount of money you have available and dividing it by the 50 best valued stocks, in this case, the size of the dataframe. From there you would take that new computed portfolio size and divide it by the stock's latest Price.

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio: ")
    
    try: 
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \nTry again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [14]:
portfolio_input()

Enter the value of your portfolio: 50000


In [ ]:
position_size = float(portfolio_size)/len(QV_dataframe.index)

for row in QV_dataframe.index:
    QV_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/QV_dataframe.loc[row, 'Price'])
    
QV_dataframe

## Building a Better (and more Realistic) Value Strategy
Every valuation metric provides beneficial insight, but also has certain flaws.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In our **better** value strategy, we will filter the stocks with the lowest percentiles on the following metrics:
    
   * Price-to-earnings ratio
   * Price-to-book ratio
   *Price-to-sales ratio
   *Enterprise Value divided by Earnings before Interest, Taxes, Depreciation, and Amortization(EV/EBITDA)
   *Enterprise Value divided by Gross Profit (EV/GP)
   
 In the example below, I use Apple's stock to compute these metrics to find the best way to retrive this specific type of data

In [41]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

#Price-to earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
#Price-to-the book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']
#Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']
#Enterprise Value divided by Earnings Before Interest, Taxes, Deprecation, and Amortized (EV/EVBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


18.13431355236741

Here, I build the new Dataframe which will be abbreviated to `rv_dataframe`. **rv** stands for robust value, which is what we base our the entire Quantitative Strategy around.

In [56]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio', 
    'PE Percentile', 
    'Price-to-Book Ratio', 
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA', 
    'EV/EBITDA Percentile', 
    'EV/GP', 
    'EV/GP Percentile', 
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
        pd.Series([
        symbol,
        data[symbol]['quote']['latestPrice'],
        'N/A', 
        data[symbol]['quote']['peRatio'], 
        'N/A', 
        data[symbol]['advanced-stats']['priceToBook'], 
        'N/A', 
        data[symbol]['advanced-stats']['priceToSales'],
        'N/A',
        ev_to_ebitda, 
        'N/A', 
        ev_to_gross_profit, 
        'N/A', 
        'N/A'   
        ],
        index = rv_columns),
            ignore_index = True
        )
    

In [57]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,165.34,N/A,51.22,N/A,10,N/A,7.98,N/A,31.374350,N/A,14.707849,N/A,N/A
1,AAL,19.53,N/A,-3.97,N/A,-1.7,N/A,0.5119,N/A,-7.437555,N/A,1.493466,N/A,N/A
2,AAP,238.31,N/A,25.05,N/A,4.43,N/A,1.37,N/A,13.166022,N/A,3.073938,N/A,N/A
3,AAPL,153.31,N/A,30.23,N/A,38.53,N/A,7.41,N/A,22.778892,N/A,17.618474,N/A,N/A
4,ABBV,112.30,N/A,29.62,N/A,15.5,N/A,3.74,N/A,10.682114,N/A,7.523886,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.71,N/A,29.94,N/A,-4.91,N/A,6.02,N/A,22.364027,N/A,10.722959,N/A,N/A
501,ZBH,157.60,N/A,36,N/A,2.6,N/A,4.11,N/A,20.527520,N/A,6.878254,N/A,N/A
502,ZBRA,553.41,N/A,38.98,N/A,11.15,N/A,5.59,N/A,28.546996,N/A,11.627541,N/A,N/A
503,ZION,68.53,N/A,6.83,N/A,1.44,N/A,3.08,N/A,5.082523,N/A,2.781947,N/A,N/A


## Dealing with Missing Data in our DataFrame
The dataframe has missing data because all the metrics that are required are not available through the API that is used.

To identify missing data, you can use panda's `isnull`

In [62]:
 rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
38,AON,323.72,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
70,BRK.B,300.16,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.51,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
188,FOX,39.25,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
202,GOOG,2814.99,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
342,NWS,25.59,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
442,TROW,213.61,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
453,UA,18.33,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

   * Drop missing data from the data set (pandas' `dropna` method is useful here)
   * Replace missing data with a new value (pandas' `fillna` method is useful here)

We will replace missing data with the **average non `NaN` data point** from that column

In [66]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now running the statement from before to identify any missing data in the table, nothing should be returned.

In [64]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles
Calculating **Value Score Percentiles** for every stock in the universe will provide the tools to compute the RV (Robust Value) Score, which will be done next. Currently, we will calcualte percentile scores for the following metrics for every stock: 

   * Price-to-earnings Ratio
   * Price-to-book ratio
   * Price-to-sales ratio
   * EV/EBITDA
   * EV/GP
   
This is how it will be calculated.

In [73]:
# Created a dictionary of metrics where every key in the dictionary is one the valuation metrics, and every value
# is the corresponding percentile column of the Pandas dataframe
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile', 
    'Price-to-Book Ratio':'PB Percentile', 
    'Price-to-Sales Ratio': 'PS Percentile', 
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

# First for loop, loops through every metric in metric dictionary
for metric in metrics.keys():
    #Second for loop, loops through every row in Pandas dataframe
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] = stats.percentileofscore( rv_dataframe[metric], rv_dataframe.loc[row,metric])
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,165.34,N/A,51.22,82.1782,10.00,78.2178,7.9800,79.802,31.374350,86.1386,14.707849,80.5941,N/A
1,AAL,19.53,N/A,-3.97,7.32673,-1.70,4.9505,0.5119,2.9703,-7.437555,2.37624,1.493466,5.74257,N/A
2,AAP,238.31,N/A,25.05,46.1386,4.43,53.4653,1.3700,17.0297,13.166022,38.6139,3.073938,13.6634,N/A
3,AAPL,153.31,N/A,30.23,58.0198,38.53,95.6436,7.4100,77.8218,22.778892,70.495,17.618474,86.3366,N/A
4,ABBV,112.30,N/A,29.62,56.8317,15.50,87.3267,3.7400,50.9901,10.682114,27.3267,7.523886,46.5347,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.71,N/A,29.94,57.4257,-4.91,4.55446,6.0200,70,22.364027,69.901,10.722959,66.1386,N/A
501,ZBH,157.60,N/A,36.00,65.8416,2.60,33.4653,4.1100,55.6436,20.527520,64.5545,6.878254,40.9901,N/A
502,ZBRA,553.41,N/A,38.98,71.0891,11.15,80.9901,5.5900,66.7327,28.546996,81.5842,11.627541,70.495,N/A
503,ZION,68.53,N/A,6.83,9.30693,1.44,12.8713,3.0800,43.7624,5.082523,4.75248,2.781947,12.0792,N/A


 ## Calculating the RV Score
Calcuating this Value Score is the most important part of this project as it will dictate what are the most optimal stocks to invest in. 

The RV Score will the arithmetic mean of the 4 percentile scores (Price-to-Earnings Ratio, Price-to-Book Ratio, Price-to-Sales Ratiom, EV/EBITDA, and EV/GP)

To calculate arithmetic mean, I will import the mean function from Python's built-in statistics module.

In [77]:
from statistics import mean

for row in rv_dataframe.index: 
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,165.34,N/A,51.22,82.1782,10.00,78.2178,7.9800,79.802,31.374350,86.1386,14.707849,80.5941,81.3861
1,AAL,19.53,N/A,-3.97,7.32673,-1.70,4.9505,0.5119,2.9703,-7.437555,2.37624,1.493466,5.74257,4.67327
2,AAP,238.31,N/A,25.05,46.1386,4.43,53.4653,1.3700,17.0297,13.166022,38.6139,3.073938,13.6634,33.7822
3,AAPL,153.31,N/A,30.23,58.0198,38.53,95.6436,7.4100,77.8218,22.778892,70.495,17.618474,86.3366,77.6634
4,ABBV,112.30,N/A,29.62,56.8317,15.50,87.3267,3.7400,50.9901,10.682114,27.3267,7.523886,46.5347,53.802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.71,N/A,29.94,57.4257,-4.91,4.55446,6.0200,70,22.364027,69.901,10.722959,66.1386,53.604
501,ZBH,157.60,N/A,36.00,65.8416,2.60,33.4653,4.1100,55.6436,20.527520,64.5545,6.878254,40.9901,52.099
502,ZBRA,553.41,N/A,38.98,71.0891,11.15,80.9901,5.5900,66.7327,28.546996,81.5842,11.627541,70.495,74.1782
503,ZION,68.53,N/A,6.83,9.30693,1.44,12.8713,3.0800,43.7624,5.082523,4.75248,2.781947,12.0792,16.5545


## Selecting the 50 Best Value Stocks

In [86]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.800,N/A,8.17,11.9802,0.5364,5.14851,0.4320,1.58416,3.645725,3.56436,0.430001,0.792079,4.61386
1,AAL,19.530,N/A,-3.97,7.32673,-1.7000,4.9505,0.5119,2.9703,-7.437555,2.37624,1.493466,5.74257,4.67327
2,PRU,116.090,N/A,6.39,8.91089,0.7081,5.54455,0.6880,4.75248,4.984503,4.35644,0.682536,1.18812,4.9505
3,OGN,38.110,N/A,2.96,8.11881,1.3000,11.0891,0.9897,10.099,2.176432,2.77228,1.455994,5.14851,7.44554
4,MCK,216.010,N/A,-7.62,6.13861,-61.3800,1.38614,0.1372,0.594059,8.416777,17.6238,2.961995,12.6733,7.68317
5,ALL,135.000,N/A,10.32,15.0495,1.4700,13.3663,0.7952,6.93069,2.532468,3.16832,0.795706,1.58416,8.0198
6,L,58.600,N/A,10.08,14.3564,0.8595,6.93069,1.0000,10.6931,5.643948,6.13861,0.993913,2.57426,8.13861
7,BA,221.690,N/A,-14.66,4.9505,-7.6600,4.35644,2.0500,30.495,-29.952113,0.792079,-208.563398,0.39604,8.19802
8,LNC,77.170,N/A,10.60,15.8416,0.6802,5.34653,0.7312,5.74257,7.984720,16.0396,0.715915,1.38614,8.87129
9,GM,58.030,N/A,6.96,9.70297,1.6300,17.3267,0.6027,4.15842,2.630755,3.36634,2.405416,10.8911,9.08911


## Calculating the Number of Shares to Buy

Make a call to the `portfolio_input` function that allows us to grab the user's portfolio size information. Based upon this, we can calculate the number of shares to buy out of the 50 best selected stocks, according to the Quantitative Value Strategy.

In [87]:
portfolio_input()

Enter the value of your portfolio: 2500000


In [95]:
position_size = float(portfolio_size) / len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.800,1736,8.17,11.9802,0.5364,5.14851,0.4320,1.58416,3.645725,3.56436,0.430001,0.792079,4.61386
1,AAL,19.530,2560,-3.97,7.32673,-1.7000,4.9505,0.5119,2.9703,-7.437555,2.37624,1.493466,5.74257,4.67327
2,PRU,116.090,430,6.39,8.91089,0.7081,5.54455,0.6880,4.75248,4.984503,4.35644,0.682536,1.18812,4.9505
3,OGN,38.110,1311,2.96,8.11881,1.3000,11.0891,0.9897,10.099,2.176432,2.77228,1.455994,5.14851,7.44554
4,MCK,216.010,231,-7.62,6.13861,-61.3800,1.38614,0.1372,0.594059,8.416777,17.6238,2.961995,12.6733,7.68317
5,ALL,135.000,370,10.32,15.0495,1.4700,13.3663,0.7952,6.93069,2.532468,3.16832,0.795706,1.58416,8.0198
6,L,58.600,853,10.08,14.3564,0.8595,6.93069,1.0000,10.6931,5.643948,6.13861,0.993913,2.57426,8.13861
7,BA,221.690,225,-14.66,4.9505,-7.6600,4.35644,2.0500,30.495,-29.952113,0.792079,-208.563398,0.39604,8.19802
8,LNC,77.170,647,10.60,15.8416,0.6802,5.34653,0.7312,5.74257,7.984720,16.0396,0.715915,1.38614,8.87129
9,GM,58.030,861,6.96,9.70297,1.6300,17.3267,0.6027,4.15842,2.630755,3.36634,2.405416,10.8911,9.08911
